In [2]:
import pandas as pd
from collections import Counter as ctr

In [3]:
#make a dataframe ignoring the last one from train.txt
train = pd.read_csv('train.txt', sep=' ', usecols=[0,1], names=['word','pos'] )
train['word'] = train.word.str.lower()
train.head(10)

,word,pos
0,confidence,NN
1,in,IN
2,the,DT
3,pound,NN
4,is,VBZ
5,widely,RB
6,expected,VBN
7,to,TO
8,take,VB
9,another,DT


In [32]:
train['ppos'] = train.pos.shift(1)
train['ppos'] = train['ppos'].fillna('O')
smooth = 0.000001
train.head()
#print(train[train['word'] == 'however'])

,word,pos,ppos
0,confidence,NN,O
1,in,IN,NN
2,the,DT,IN
3,pound,NN,DT
4,is,VBZ,NN


### Write the functions
$P(T{i}|T{i-1})$
$P(W{i}|T{i})$

In [48]:
# prev_pos contains a dict for every ppos in it... shows the amount of times each pos follows the ppos(key) 
#where every pos is also a key within ppos
prev_pos = {}
prev_pos_count = {} #count of how many times a ppos shows up
for pPos in set(train.ppos):
    temp_df = train[train.ppos == pPos] #dataframe with all instances of given ppos
    counted = ctr(temp_df.pos) #dict with counts for each pos following ppos
    prev_pos[pPos] = counted
    prev_pos_count[pPos] = len(temp_df)
    
def Ptt(Ti='',Tprev=''):
    if Ti not in prev_pos[Tprev]: return smooth
    return prev_pos[Tprev][Ti] / prev_pos_count[Tprev]

In [49]:
word_pos = {} #dict where every pos(key)'s value is the counted dict
word_pos_count = {} #dict with count of the times every pos is in train df
for pos in set(train['pos']):
    temp_df = train[train['pos'] == pos]
    counted = ctr(temp_df.word) #every word that belongs to pos with their # of ocurrences
    word_pos[pos] = counted
    word_pos_count[pos] = len(temp_df)

def Pwt(W='', T=''):
    if W not in word_pos[T]: return smooth
    return word_pos[T][W] / word_pos_count[T]

In [50]:
Pwt('pound','NN')

0.00046439115003151226

## Build the trellis

In [58]:
sequence = ['this', 'is', 'not', 'the', 'time', 'for', 'surrender']

#list of tuples where there first position is the pos and second is the P of the word given the pos
emissions = []

#list of tuple where the first position is the ppos and the second the P of pos given ppos
transitions = []

for word in sequence:
    emission = [(t,Pwt(word, t)) for t in train.pos]
    transition = [(tprev,Ptt(ti,tprev)) for ti in pos for tprev,prob in emission]
    emissions.append(emission)
    transitions.append(transition)

## Evaluate: decode

In [14]:
test = pd.read_csv('test.txt', sep=' ', usecols=[0,1], names=['word','pos'])
test['word'] = test['word'].str.lower()
test.head()

,word,pos
0,rockwell,NNP
1,international,NNP
2,corp.,NNP
3,'s,POS
4,tulsa,NNP
